In [13]:
from morpholog import Morpholog 
import re
from collections import Counter, defaultdict
import pickle
import json

with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read().split()

In [14]:
class Morphology_Tok:
    def __init__(self, text):
        self.text = text

    def tokenize_with_morpholog(self):
            analyzer = Morpholog()
            roots = []
            endings = []

            for word in self.text:
                root = analyzer.get_roots(word)
                ending = analyzer.get_ending(word)

                if root:
                    cleaned_root = root[0].strip("',\"()")
                    roots.append(cleaned_root)

                if ending:
                    cleaned_ending = ending[0].strip("',\"().")
                    endings.append(cleaned_ending)

            return roots, endings
    
morph = Morphology_Tok(text)
roots, endings = morph.tokenize_with_morpholog()
for root in roots:
     print(root)


сказ
о
кандал
i
алекс
аниканиканов
кова
в
тарые
до
блеск
шлифова
на
чей
ногах
кандалы.
кова
они
бы
давно.
алекс
зна
о
эт
в
ибири.
в
двор
каторж
тюрьм
он
останов
тарик
наклонился
щуп
кандал
и
воскликнул:
–
ши
я
их
нашивал!
по
звон
узнал!
слыш
–
знаком
на
кубани
год
пят
тот
назад
таскал.
нов
бы
еще
шершавые.
до
я
они
грузин
один
гремел.
др
из
камеры
они
кинул.
тут
я
осудили
стать
я
они
впадать.
как
гн
я
в
ибирь

я
в
один
мест
в
бане
намылил
в
кровь
др
ноги
сня
кандал
и
юрк
под
лок.
не
один
я
–
тр
лепет
тогда.
эх
помин
пь
стар
и
жечь
он
туск
глаза.
он
миг
алекс
и
треп
он
по
плечу:
–

ты
кандал
на
ты
частливые!
нимаешь
частливые?!
слух
о
том
чт
стар
через
пят
год
по
звон
зна
свой
цепи
ник
в
весь
камеры.
вер
он
не
все
но
стар
се
омнения:
на
кобыл
кандал
хран
он
метка.
эт
на
весь
тюрьм
слав
стар
и
кова
вним
к
кандалам:
кто
они
думал?
кто
они
ковывает?
арестантам
чт
в
рот

дел
кандалы

и
ши
смерт
рубахи
каторг
сл
не
один
проклятие.
ин
каторж
крепк
зл
и
тосковали.
ин
одрогнулись
гля
правда
в
г

In [15]:

with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read().split()

# Класс BPE_Tokenizer
class BPE_Tokenizer:
    def __init__(self, num_merges):
        self.num_merges = num_merges
        self.vocab = None
        self.bpe_codes = defaultdict(int)

    def build_vocab(self, words):
        """ Строим словарь из списка слов """
        self.vocab = Counter()
        for word in words:
            # Добавляем маркер конца слова
            word += ' </w>'
            self.vocab[word] += 1

    def get_stats(self):
        """ Считаем статистику пар символов """
        pairs = defaultdict(int)
        for word, freq in self.vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    def merge_vocab(self, pair):
        """ Объединяем наиболее частые пары символов """
        new_vocab = Counter()
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in self.vocab:
            w_out = p.sub(''.join(pair), word)
            new_vocab[w_out] = self.vocab[word]
        self.vocab = new_vocab

    def bpe(self):
        """ Применяем BPE алгоритм """
        for i in range(self.num_merges):
            pairs = self.get_stats()
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            self.bpe_codes[best] += pairs[best]
            self.merge_vocab(best)
            print(f"Итерация {i}: Объединено {best}, частота: {pairs[best]}")
            # if pairs[best] < self.num_merges:
            #     break

# Пример использования
# num_merges = 100  # Указываем порог для объединения Ограничение по частоте: Ваш код также содержит условие, которое прерывает цикл, если частота наиболее частой пары символов меньше num_merges. Это условие может привести к тому, что цикл завершится раньше, чем будут выполнены все num_merges итераций.

# tokenizer = BPE_Tokenizer(num_merges)
# tokenizer.build_vocab(text)  # Передаем список слов вместо пути к файлу
# tokenizer.bpe()


In [16]:
class CombinedTokenizer:
    def __init__(self, text, num_merges):
        self.text = text
        self.num_merges = num_merges
        self.combined_vocab = None  # Инициализация атрибута

    def tokenize(self):
        # Инициализация и обработка текста морфологическим токенизатором
        morphology_tok = Morphology_Tok(self.text)
        roots, endings = morphology_tok.tokenize_with_morpholog()

        # Инициализация и обработка текста BPE токенизатором
        bpe_tok = BPE_Tokenizer(self.num_merges)
        bpe_tok.build_vocab(self.text)  # Предполагается, что текст уже разбит на слова
        bpe_tok.bpe()

        print(f"Корни: {roots}")
        print(f"Окончания: {endings}")
        print(f"BPE словарь: {list(bpe_tok.vocab.keys())}")

        # Объединение результатов в один словарь
        self.combined_vocab = set(roots + endings + list(bpe_tok.vocab.keys()))

        self.token_to_id = {token: id for id, token in enumerate(self.combined_vocab)}
        self.id_to_token = {id: token for token, id in self.token_to_id.items()}

        # Преобразование токенов в числовые идентификаторы
        token_ids = [self.token_to_id[token] for token in self.combined_vocab]
        return token_ids
    
    def save_vocab(self):
        # Сохранение словаря в файл
        with open('token_to_id.pkl', 'wb') as file:
            pickle.dump(self.token_to_id, file)
        print("Словарь успешно сохранен.")
        
        # Сохранение обратного словаря в файл
        with open('id_to_token.pkl', 'wb') as file:
            pickle.dump(self.id_to_token, file)
        print("Обратный словарь успешно сохранен.")

    def save_vocab_json(self):
        # Сохранение словаря token_to_id в файл JSON
        with open('token_to_id.json', 'w', encoding='utf-8') as file:
            json.dump(self.token_to_id, file, ensure_ascii=False, indent=4)
        print("Словарь token_to_id успешно сохранен в формате JSON.")

        # Сохранение обратного словаря id_to_token в файл JSON
        with open('id_to_token.json', 'w', encoding='utf-8') as file:
            json.dump(self.id_to_token, file, ensure_ascii=False, indent=4)
        print("Обратный словарь id_to_token успешно сохранен в формате JSON.")
    
def test_tokenizer(tokenizer):
    tokens = tokenizer.tokenize()
    print(f"Токены: {tokens}")
    print(f"Количество токенов: {len(tokens)}")

# Пример использования
text = "Пример текста для токенизации"
num_merges = 100  # Количество итераций для BPE
tokenizer = CombinedTokenizer(text, num_merges)
token_ids = tokenizer.tokenize()

# Сохранение словарей
tokenizer.save_vocab()
tokenizer.save_vocab_json()

# Сохранение экземпляра токенизатора в файл
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
print("Токенизатор успешно сохранен.")

Итерация 0: Объединено ('и', '</w>'), частота: 4
Итерация 1: Объединено ('е', '</w>'), частота: 3
Итерация 2: Объединено ('т', '</w>'), частота: 3
Итерация 3: Объединено ('р', '</w>'), частота: 2
Итерация 4: Объединено ('к', '</w>'), частота: 2
Итерация 5: Объединено ('а', '</w>'), частота: 2
Итерация 6: Объединено ('П', '</w>'), частота: 1
Итерация 7: Объединено ('м', '</w>'), частота: 1
Итерация 8: Объединено ('с', '</w>'), частота: 1
Итерация 9: Объединено ('д', '</w>'), частота: 1
Итерация 10: Объединено ('л', '</w>'), частота: 1
Итерация 11: Объединено ('я', '</w>'), частота: 1
Итерация 12: Объединено ('о', '</w>'), частота: 1
Итерация 13: Объединено ('н', '</w>'), частота: 1
Итерация 14: Объединено ('з', '</w>'), частота: 1
Итерация 15: Объединено ('ц', '</w>'), частота: 1
Корни: ['п', 'р', 'и', 'м', 'е', 'р', 'так', 'е', 'к', 'с', 'так', 'дал', 'л', 'я', 'так', 'о', 'к', 'е', 'и', 'з', 'ц', 'и', 'и']
Окончания: ['+', '+', '+', '+', '+', '+']
BPE словарь: ['П</w>', 'р</w>', 'и</w